# Tensorflow MNIST Model

 * Wrap a Tensorflow MNIST python model for use as a prediction microservice in seldon-core
 
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Dependencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)

```bash
pip install seldon-core
```

## Train locally
 

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)
import tensorflow as tf

if __name__ == '__main__':
    
    x = tf.placeholder(tf.float32, [None,784], name="x")

    W = tf.Variable(tf.zeros([784,10]))
    b = tf.Variable(tf.zeros([10]))

    y = tf.nn.softmax(tf.matmul(x,W) + b, name="y")

    y_ = tf.placeholder(tf.float32, [None, 10])


    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

    init = tf.initialize_all_variables()

    sess = tf.Session()
    sess.run(init)

    for i in range(1000):
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict = {x: mnist.test.images, y_:mnist.test.labels}))

    saver = tf.train.Saver()

    saver.save(sess, "model/deep_mnist_model")



Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.918


Wrap model using s2i

In [2]:
!s2i build . seldonio/seldon-core-s2i-python36:0.10 deep-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
Looking in links: /whl
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [3]:
!docker run --name "mnist_predictor" -d --rm -p 5000:5000 deep-mnist:0.1

d8676abb76c33c3ba10300a3625e242b405b3bad70d9b74be5ce8addd5e44ffd


Send some random features that conform to the contract

In [4]:
!seldon-core-tester contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:

[[0.733 0.176 0.261 0.111 0.974 0.635 0.182 0.747 0.297 0.582 0.739 0.531
  0.216 0.977 0.586 0.443 0.33  0.423 0.772 0.923 0.534 0.891 0.056 0.437
  0.565 0.257 0.459 0.384 0.265 0.902 0.646 0.49  0.334 0.769 0.794 0.097
  0.09  0.543 0.711 0.635 0.919 0.402 0.969 0.452 0.134 0.211 0.773 0.201
  0.953 0.424 0.797 0.852 0.041 0.101 0.91  0.95  0.479 0.708 0.941 0.721
  0.855 0.509 0.818 0.592 0.908 0.827 0.163 0.187 0.405 0.874 0.73  0.719
  0.051 0.807 0.019 0.645 0.774 0.304 0.506 0.461 0.694 0.28  0.197 0.031
  0.213 0.211 0.987 0.175 0.641 0.19  0.446 0.776 0.892 0.755 0.266 0.803
  0.384 0.698 0.328 0.952 0.515 0.093 0.616 0.942 0.848 0.088 0.028 0.858
  0.119 0.177 0.463 0.798 0.461 0.011 0.601 0.397 0.02  0.064 0.309 0.757
  0.911 0.06  0.931 0.997 0.85  0.651 0.731 0.071 0.63  0.137 0.821 0.854
  0.726 0.902 0.532 0.032 0.154 0.294 0.307 0.039 0.976 0.757 0.444 0.911
  0.732 0.029 0.944 0.238 0.437 0.927 0.482 0.844

In [5]:
!docker rm mnist_predictor --force

mnist_predictor


## Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need [s2i >= 1.1.13](https://github.com/openshift/source-to-image/releases/tag/v1.1.13)**

In [6]:
!minikube start --memory 4096

😄  minikube v0.34.1 on linux (amd64)
🔥  Creating virtualbox VM (CPUs=2, Memory=4096MB, Disk=20000MB) ...
📶  "minikube" IP address is 192.168.99.100
🐳  Configuring Docker as the container runtime ...
✨  Preparing Kubernetes environment ...
🚜  Pulling images required by Kubernetes v1.13.3 ...
🚀  Launching Kubernetes v1.13.3 using kubeadm ... 
🔑  Configuring cluster permissions ...
🤔  Verifying component health .....
💗  kubectl is now configured to use "minikube"
🏄  Done! Thank you for using minikube!


In [7]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [8]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [9]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [6]:
!helm install ../../../helm-charts/seldon-core-operator --name seldon-core --set usageMetrics.enabled=true --namespace seldon-system

NAME:   seldon-core
LAST DEPLOYED: Thu Apr 25 09:13:58 2019
NAMESPACE: seldon-system
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1/ClusterRole
seldon-operator-manager-role  0s

==> v1/ClusterRoleBinding
NAME                                 AGE
seldon-operator-manager-rolebinding  0s

==> v1/Service
NAME                                        TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)  AGE
seldon-operator-controller-manager-service  ClusterIP  10.109.84.44  <none>       443/TCP  0s

==> v1/StatefulSet
NAME                                DESIRED  CURRENT  AGE
seldon-operator-controller-manager  1        1        0s

==> v1/Pod(related)
NAME                                  READY  STATUS             RESTARTS  AGE
seldon-operator-controller-manager-0  0/1    ContainerCreating  0         0s

==> v1/Secret
NAME                                   TYPE    DATA  AGE
sel

In [7]:
!kubectl rollout status statefulset.apps/seldon-operator-controller-manager -n seldon-system

partitioned roll out complete: 1 new pods have been updated...


## Setup Ingress
There are gRPC issues with the latest Ambassador, so we rewcommend 0.40.2 until these are fixed.

In [8]:
!helm install stable/ambassador --name ambassador --set crds.keep=false

NAME:   ambassador
LAST DEPLOYED: Thu Apr 25 09:14:31 2019
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME        SECRETS  AGE
ambassador  1        0s

==> v1beta1/ClusterRole
NAME        AGE
ambassador  0s

==> v1beta1/ClusterRoleBinding
NAME        AGE
ambassador  0s

==> v1/Service
NAME               TYPE          CLUSTER-IP     EXTERNAL-IP  PORT(S)                     AGE
ambassador-admins  ClusterIP     10.110.99.128  <none>       8877/TCP                    0s
ambassador         LoadBalancer  10.97.7.72     <pending>    80:30064/TCP,443:32402/TCP  0s

==> v1/Deployment
NAME        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
ambassador  3        3        3           0          0s

==> v1/Pod(related)
NAME                         READY  STATUS             RESTARTS  AGE
ambassador-5b89d44544-5hhl7  0/1    ContainerCreating  0         0s
ambassador-5b89d44544-5xcdw  0/1    ContainerCreating  0         0s
ambassador-5b89d44544-7rv6r  0/1    ContainerCreatin

In [9]:
!kubectl rollout status deployment.apps/ambassador

Waiting for deployment "ambassador" rollout to finish: 0 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 1 of 3 updated replicas are available...
Waiting for deployment "ambassador" rollout to finish: 2 of 3 updated replicas are available...
deployment "ambassador" successfully rolled out


## Wrap Model and Test

In [10]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-python2:0.5.1 deep-mnist:0.1

---> Installing application source...
---> Installing dependencies ...
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
Looking in links: /whl
Url '/whl' is ignored. It is either a non-existing path or lacks a specific scheme.
You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Build completed successfully


In [11]:
!kubectl create -f deep_mnist.json

seldondeployment.machinelearning.seldon.io/deep-mnist created


In [12]:
!kubectl rollout status deploy/deep-mnist-single-model-8969cc0

Waiting for deployment "deep-mnist-single-model-8969cc0" rollout to finish: 0 of 1 updated replicas are available...
deployment "deep-mnist-single-model-8969cc0" successfully rolled out


In [13]:
!seldon-core-api-tester contract.json `minikube ip` `kubectl get svc ambassador -o jsonpath='{.spec.ports[0].nodePort}'` \
    deep-mnist --namespace default -p

----------------------------------------
SENDING NEW REQUEST:

[[0.07  0.525 0.195 0.946 0.425 0.312 0.099 0.855 0.955 0.769 0.156 0.647
  0.479 0.197 0.586 0.616 0.105 0.862 0.073 0.335 0.277 0.345 0.872 0.247
  0.266 0.289 0.396 0.217 0.143 0.685 0.567 0.425 0.919 0.474 0.436 0.6
  0.341 0.776 0.417 0.541 0.62  0.161 0.164 0.757 0.135 0.982 0.491 0.735
  0.837 0.387 0.628 0.069 0.062 0.73  0.742 0.563 0.22  0.964 0.01  0.084
  0.681 0.553 0.746 0.834 0.143 0.34  0.676 0.794 0.562 0.113 0.195 0.309
  0.334 0.45  0.936 0.233 0.435 0.105 0.347 0.149 0.378 0.939 0.844 0.912
  0.869 0.251 0.231 0.596 0.603 0.716 0.086 0.669 0.78  0.265 0.316 0.063
  0.296 0.347 0.23  0.843 0.031 0.923 0.978 0.623 0.738 0.362 0.186 0.905
  0.138 0.952 0.209 0.218 0.407 0.198 0.489 0.838 0.372 0.335 0.908 0.505
  0.551 0.256 0.966 0.827 0.121 0.642 0.321 0.949 0.225 0.903 0.954 0.193
  0.378 0.109 0.684 0.026 0.804 0.108 0.104 0.646 0.101 0.097 0.303 0.528
  0.49  0.91  0.523 0.868 0.22  0.555 0.353 0.627 0

In [16]:
!minikube delete

🔥  Deleting "minikube" from virtualbox ...
💔  The "minikube" cluster has been deleted.
